In [ ]:
# Data.Qubit
# Data.Int.Util
# Language.Quil.Types
# Language.Quil.Tests

In [1]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

In [212]:
import Control.Arrow ((&&&), (***), second)
import Data.Foldable
import Data.Complex
import Data.List
import Numeric.LinearAlgebra.Array
import Numeric.LinearAlgebra.Array.Util
import Numeric.LinearAlgebra.Tensor
import qualified Data.Vector.Storable as V

In [3]:
isqrt :: Int -> Int
isqrt n = snd . head . dropWhile ((< n) . fst) $ ((^2) &&& id) <$> [0..]

In [5]:
ilog2 :: Int -> Int
ilog2 n = snd . head . dropWhile ((< n) . fst) $ ((2^) &&& id) <$> [0..]

In [7]:
ilog2sqrt :: Int -> Int
ilog2sqrt n = snd . head . dropWhile ((< n) . fst) $ ((^2) . (2^) &&& id) <$> [0..]

In [37]:
type Qubit = Int
type Amplitude = Complex Double
newtype Wavefunction = Wavefunction {wavefunction :: Tensor Amplitude}
  deriving (Eq, Floating, Fractional, Num)

In [220]:
instance Show Wavefunction where
  show =
    showTensor wavefunction
      $ \_ (k, v) ->
      showAmplitude v ++ "|" ++ concatMap show k ++ ">"

In [68]:
indexPrefix :: Variant -> Char
indexPrefix Contra = 'm'
indexPrefix Co     = 'n'
indexLabels :: Variant -> [Qubit] -> [(String, String)]
indexLabels variant qubits =
  let
    n = length qubits
    o =
      case variant of
        Contra -> 0
        Co     -> n
  in
    zipWith (\k v -> (show k, indexPrefix variant : show v)) [(1+o)..] qubits
wavefunctionLabels :: Int -> [(String, String)]
wavefunctionLabels n = indexLabels Contra [0..(n-1)]
operatorLabels :: [Qubit] -> [(String, String)]
operatorLabels qubits = indexLabels Contra qubits ++ indexLabels Co qubits

In [69]:
toWavefunction :: [Amplitude] -> Wavefunction
toWavefunction amplitudes =
  let
    n = ilog2 $ length amplitudes
  in
    Wavefunction
      . renameExplicit (wavefunctionLabels n)
      $ listTensor (replicate n 2) amplitudes
groundState :: Qubit -> Wavefunction
groundState = toWavefunction . (1 :) . (`replicate` 0) . (+ (-1)) . (2^)

In [71]:
newtype Operator = Operator {operator :: Tensor (Complex Double)}
  deriving (Eq, Floating, Fractional, Num)

In [218]:
showAmplitude :: Amplitude -> String
showAmplitude (a :+ b)
  | b == 0    = show a
  | a == 0    = show b ++ "j"
  | otherwise = "(" ++ show a ++ "+" ++ show b ++ "j)"

In [221]:
showTensor :: (a -> Tensor Amplitude) -> (Int -> ([Int], Amplitude) -> String) -> a -> String
showTensor toTensor format x =
    let
      x' = toTensor x
      n = order x'
    in
      (++ (filter (/= '"') . show . fmap tail . filter ((== indexPrefix Contra) . head) $ names x'))
      . (++ " @ ")
      . intercalate " + "
        . fmap (format n)
        . filter ((/= 0) . snd)
        . zip (fmap reverse . sequence $ replicate n [0, 1])
        . V.toList
        $ coords x'

In [246]:
instance Show Operator where
  show =
    showTensor operator
      $ \n (k, v) ->
      let
        (k0, k1) = splitAt (n `div` 2) $ concatMap show k
      in
        showAmplitude v ++ "|" ++ k1 ++ "><" ++ k0 ++ "|"

In [73]:
asOperator :: [Qubit] -> [Amplitude] -> Operator
asOperator qubits amplitudes =
  let
    n = length qubits
  in
    Operator
      . renameExplicit (operatorLabels qubits)
      $ listTensor (replicate n 2 ++ replicate n (-2)) amplitudes

In [120]:
mult :: Tensor Amplitude -> Tensor Amplitude -> Tensor Amplitude
mult x y =
  let
    lft  = filter ((== 'n') . head) $ names x
    rght = filter ((== 'm') . head) $ names y
    cmmn = fmap tail lft `intersect` fmap tail rght
    x' = renameExplicit [(i, 'p' : tail i) | i <- filter ((`elem` cmmn) . tail) lft ] x
    y' = renameExplicit [(i, 'p' : tail i) | i <- filter ((`elem` cmmn) . tail) rght] y
  in
    x' * y' 

In [177]:
(.*.) :: Operator -> Operator -> Operator
Operator x .*. Operator y = Operator $ x `mult` y
(.*) :: Operator -> Wavefunction -> Wavefunction
Operator x .* Wavefunction y = Wavefunction $ x `mult` y
(..*) :: Foldable t => t Operator -> Wavefunction -> Wavefunction
(..*) = flip . foldl $ flip (.*)

In [233]:
data Gate =
    I Qubit
  | X Qubit
  | Y Qubit
  | Z Qubit
  | H Qubit
  | PHASE Double Qubit
  | S Qubit
  | T Qubit
  | CPHASE00 Double Qubit Qubit
  | CPHASE01 Double Qubit Qubit
  | CPHASE10 Double Qubit Qubit
  | CPHASE Double Qubit Qubit
  | RX Double Qubit
  | RY Double Qubit
  | RZ Double Qubit
  | CNOT Qubit Qubit
  | CCNOT Qubit Qubit Qubit 
  | PSWAP Double Qubit Qubit
  | SWAP Qubit Qubit
  | ISWAP Qubit Qubit
  | CSWAP Qubit Qubit Qubit
  | CZ Qubit Qubit
  deriving (Eq, Ord, Read, Show)

In [234]:
fromGate :: Gate -> Operator

fromGate (I i) =
  asOperator [i]
    [
      1, 0
    , 0, 1
    ]

fromGate (X i) =
  asOperator [i]
    [
      0, 1
    , 1, 0
    ]

fromGate (Y i) =
  asOperator [i]
    [
      0     , 0 :+ (-1)
    , 0 :+ 1, 0
    ]

fromGate (Z i) =
  asOperator [i]
    [
      1, 0
    , 0, -1
    ]

fromGate (H i) =
  asOperator [i]
    [
      1, 1
    , 1, -1
    ] / sqrt 2

fromGate (PHASE theta i) =
  asOperator [i]
    [
      1, 0
    , 0, cis theta
    ]

fromGate (S i) =
  asOperator [i]
    [
      1, 0
    , 0, 0 :+ 1
    ]

fromGate (T i) =
  asOperator [i]
    [
      1, 0
    , 0, (1 :+ 1) / sqrt 2
    ]

fromGate (CPHASE00 theta i j) =
  asOperator [i, j]
    [
      cis theta, 0, 0, 0
    , 0        , 1, 0, 0
    , 0        , 0, 1, 0
    , 0        , 0, 0, 1
    ]

fromGate (CPHASE01 theta i j) =
  asOperator [i, j]
    [
      1, 0        , 0, 0
    , 0, cis theta, 0, 0
    , 0, 0        , 1, 0
    , 0, 0        , 0, 1
    ]

fromGate (CPHASE10 theta i j) =
  asOperator [i, j]
    [
      1, 0, 0        , 0
    , 0, 1, 0        , 0
    , 0, 0, cis theta, 0
    , 0, 0, 0        , 1
    ]

fromGate (CPHASE theta i j) =
  asOperator [i, j]
    [
      1, 0, 0, 0
    , 0, 1, 0, 0
    , 0, 0, 1, 0
    , 0, 0, 0, cis theta
    ]

fromGate (RX theta i) =
  asOperator [i]
    [
      cos (theta / 2) :+ 0        , 0 :+ (- sin (theta / 2))
    , 0 :+ (- sin (theta / 2)), cos (theta / 2) :+ 0
    ]

fromGate (RY theta i) =
  asOperator [i]
    [
      cos (theta / 2) :+ 0, (- sin (theta / 2)) :+ 0
    , sin (theta / 2) :+ 0, cos (theta / 2) :+ 0
    ]

fromGate (RZ theta i) =
  asOperator [i]
    [
      cis (- theta / 2), 0
    , 0                , cis (theta / 2)
    ]

fromGate (CNOT i j) =
  asOperator [i, j]
    [
      1, 0, 0, 0
    , 0, 1, 0, 0
    , 0, 0, 0, 1
    , 0, 0, 1, 0
    ]

fromGate (CCNOT i j k) =
  asOperator [i, j, k]
    [
      1, 0, 0, 0, 0, 0, 0, 0
    , 0, 1, 0, 0, 0, 0, 0, 0
    , 0, 0, 1, 0, 0, 0, 0, 0
    , 0, 0, 0, 1, 0, 0, 0, 0
    , 0, 0, 0, 0, 1, 0, 0, 0
    , 0, 0, 0, 0, 0, 1, 0, 0
    , 0, 0, 0, 0, 0, 0, 0, 1
    , 0, 0, 0, 0, 0, 0, 1, 0
    ]
    
fromGate (PSWAP theta i j) =
  asOperator [i, j]
    [
      1, 0        , 0        , 0
    , 0, 0        , cis theta, 0
    , 0, cis theta, 0        , 0
    , 0, 0        , 0        , 1
    ]
    
fromGate (SWAP i j) =
  asOperator [i, j]
    [
      1, 0, 0, 0
    , 0, 0, 1, 0
    , 0, 1, 0, 0
    , 0, 0, 0, 1
    ]

fromGate (ISWAP i j) =
  asOperator [i, j]
    [
      1, 0     , 0     , 0
    , 0, 0     , 0 :+ 1, 0
    , 0, 0 :+ 1, 0     , 0
    , 0, 0     , 0     , 1
    ]
    
fromGate (CSWAP i j k) =
  asOperator [i, j, k]
    [
      1, 0, 0, 0, 0, 0, 0, 0
    , 0, 1, 0, 0, 0, 0, 0, 0
    , 0, 0, 1, 0, 0, 0, 0, 0
    , 0, 0, 0, 1, 0, 0, 0, 0
    , 0, 0, 0, 0, 1, 0, 0, 0
    , 0, 0, 0, 0, 0, 0, 1, 0
    , 0, 0, 0, 0, 0, 1, 0, 0
    , 0, 0, 0, 0, 0, 0, 0, 1
    ]
    
fromGate (CZ i j) =
  asOperator [i, j]
    [
      1, 0, 0, 0
    , 0, 1, 0, 0
    , 0, 0, 1, 0
    , 0, 0, 0, -1
    ]

In [235]:
theta = 0.54321
sequence_
  [
    do
      putStrLn l
      sequence_
        [
          putStrLn $ show (fromGate (o 0) .* e) ++ " : " ++ s
        |
          (s, e) <- second toWavefunction  <$> [("<0|", [1,0]), ("<1", [0,1])]
        ]
  |
    (l, o) <- [
                ("I", I)
              , ("X", X)
              , ("Y", Y)
              , ("Z", Z)
              , ("H", H)
              , ("PHASE", PHASE theta)
              , ("S", S)
              , ("T", T)
              , ("RX", RX theta)
              , ("RY", RY theta)
              , ("RZ", RZ theta)
              ]
  ]

I
1.0|0> @ [0] : <0|
1.0|1> @ [0] : <1
X
1.0|1> @ [0] : <0|
1.0|0> @ [0] : <1
Y
1.0j|1> @ [0] : <0|
-1.0j|0> @ [0] : <1
Z
1.0|0> @ [0] : <0|
-1.0|1> @ [0] : <1
H
0.7071067811865475|0> + 0.7071067811865475|1> @ [0] : <0|
0.7071067811865475|0> + -0.7071067811865475|1> @ [0] : <1
PHASE
1.0|0> @ [0] : <0|
(0.856053888710673+0.516886582939947j)|1> @ [0] : <1
S
1.0|0> @ [0] : <0|
1.0j|1> @ [0] : <1
T
1.0|0> @ [0] : <0|
(0.7071067811865475+0.7071067811865475j)|1> @ [0] : <1
RX
0.9633415512451108|0> + -0.2682779447600259j|1> @ [0] : <0|
-0.2682779447600259j|0> + 0.9633415512451108|1> @ [0] : <1
RY
0.9633415512451108|0> + 0.2682779447600259|1> @ [0] : <0|
-0.2682779447600259|0> + 0.9633415512451108|1> @ [0] : <1
RZ
(0.9633415512451108+-0.2682779447600259j)|0> @ [0] : <0|
(0.9633415512451108+0.2682779447600259j)|1> @ [0] : <1

In [236]:
sequence_
  [
    do
      putStrLn l
      sequence_
        [
          putStrLn $ show (fromGate (o 0 1) .* e) ++ " : " ++ s
        |
          (s, e) <- second toWavefunction  <$> [
                                                 ("<00|", [1,0,0,0])
                                               , ("<10|", [0,1,0,0])
                                               , ("<01|", [0,0,1,0])
                                               , ("<11|", [0,0,0,1])
                                               ]
        ]
  |
    (l, o) <- [
                ("CPHASE00", CPHASE00 theta)
              , ("CPHASE01", CPHASE01 theta)
              , ("CPHASE10", CPHASE10 theta)
              , ("CPHASE", CPHASE theta)
              , ("CNOT", CNOT)
              , ("PSWAP", PSWAP theta)
              , ("SWAP", SWAP)
              , ("ISWAP", ISWAP)
              , ("CZ", CZ)
              ]
  ]

CPHASE00
(0.856053888710673+0.516886582939947j)|00> @ [0,1] : <00|
1.0|10> @ [0,1] : <10|
1.0|01> @ [0,1] : <01|
1.0|11> @ [0,1] : <11|
CPHASE01
1.0|00> @ [0,1] : <00|
(0.856053888710673+0.516886582939947j)|10> @ [0,1] : <10|
1.0|01> @ [0,1] : <01|
1.0|11> @ [0,1] : <11|
CPHASE10
1.0|00> @ [0,1] : <00|
1.0|10> @ [0,1] : <10|
(0.856053888710673+0.516886582939947j)|01> @ [0,1] : <01|
1.0|11> @ [0,1] : <11|
CPHASE
1.0|00> @ [0,1] : <00|
1.0|10> @ [0,1] : <10|
1.0|01> @ [0,1] : <01|
(0.856053888710673+0.516886582939947j)|11> @ [0,1] : <11|
CNOT
1.0|00> @ [0,1] : <00|
1.0|10> @ [0,1] : <10|
1.0|11> @ [0,1] : <01|
1.0|01> @ [0,1] : <11|
PSWAP
1.0|00> @ [0,1] : <00|
(0.856053888710673+0.516886582939947j)|01> @ [0,1] : <10|
(0.856053888710673+0.516886582939947j)|10> @ [0,1] : <01|
1.0|11> @ [0,1] : <11|
SWAP
1.0|00> @ [0,1] : <00|
1.0|01> @ [0,1] : <10|
1.0|10> @ [0,1] : <01|
1.0|11> @ [0,1] : <11|
ISWAP
1.0|00> @ [0,1] : <00|
1.0j|01> @ [0,1] : <10|
1.0j|10> @ [0,1] : <01|
1.0|11> @ [0,1] : <

In [237]:
sequence_
  [
    do
      putStrLn l
      sequence_
        [
          putStrLn $ show (fromGate (o 0 1 2) .* e) ++ " : " ++ s
        |
          (s, e) <- second toWavefunction  <$> [
                                                 ("<000|", [1,0,0,0,0,0,0,0])
                                               , ("<100|", [0,1,0,0,0,0,0,0])
                                               , ("<010|", [0,0,1,0,0,0,0,0])
                                               , ("<110|", [0,0,0,1,0,0,0,0])
                                               , ("<001|", [0,0,0,0,1,0,0,0])
                                               , ("<101|", [0,0,0,0,0,1,0,0])
                                               , ("<011|", [0,0,0,0,0,0,1,0])
                                               , ("<111|", [0,0,0,0,0,0,0,1])
                                               ]
        ]
  |
    (l, o) <- [
                ("CCNOT", CCNOT)
              , ("CSWAP", CSWAP)
              ]
  ]

CCNOT
1.0|000> @ [0,1,2] : <000|
1.0|100> @ [0,1,2] : <100|
1.0|010> @ [0,1,2] : <010|
1.0|110> @ [0,1,2] : <110|
1.0|001> @ [0,1,2] : <001|
1.0|101> @ [0,1,2] : <101|
1.0|111> @ [0,1,2] : <011|
1.0|011> @ [0,1,2] : <111|
CSWAP
1.0|000> @ [0,1,2] : <000|
1.0|100> @ [0,1,2] : <100|
1.0|010> @ [0,1,2] : <010|
1.0|110> @ [0,1,2] : <110|
1.0|001> @ [0,1,2] : <001|
1.0|011> @ [0,1,2] : <101|
1.0|101> @ [0,1,2] : <011|
1.0|111> @ [0,1,2] : <111|

In [248]:
fromGate $ CCNOT 0 1 2

1.0|000><000| + 1.0|100><100| + 1.0|010><010| + 1.0|110><110| + 1.0|001><001| + 1.0|101><101| + 1.0|011><111| + 1.0|111><011| @ [0,1,2]

In [242]:
coords . operator . fromGate $ Y 0

[0.0 :+ 0.0,0.0 :+ (-1.0),0.0 :+ 1.0,0.0 :+ 0.0]

In [243]:
names. operator . fromGate $ Y 0

["m0","n0"]

In [ ]:
# Y
#  1.0j|1> @ [0] : <0|
# -1.0j|0> @ [0] : <1|

In [ ]:
# |0><0|, |0><1|, |1><0|, |1><1|

In [250]:
asOperator [0, 1] $ (:+ 0) <$> [1..16]

1.0|00><00| + 2.0|00><10| + 3.0|00><01| + 4.0|00><11| + 5.0|10><00| + 6.0|10><10| + 7.0|10><01| + 8.0|10><11| + 9.0|01><00| + 10.0|01><10| + 11.0|01><01| + 12.0|01><11| + 13.0|11><00| + 14.0|11><10| + 15.0|11><01| + 16.0|11><11| @ [0,1]